In [1]:
# importing libraries for data modeling

import pandas as pd
import numpy as np
import collections
from nltk.tokenize.treebank import TreebankWordTokenizer
from common import utils, vocabulary
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import tensorflow as tf
import os
import time
import datetime
from CNN.text_cnn import TextCNN
from tensorflow.contrib import learn

/home/malexander92/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# loading model data created in data exploration

model_data = pd.read_csv("data/got_comments_top_10_sampled.csv")

train, test = train_test_split(model_data, test_size=0.1)

train_x = train['body']
train_y = train['author']
test_x = test['body']
test_y = test['author']

In [3]:
# tokenize data in comment body using Penn Treebank tokenizer

tokenizer = TreebankWordTokenizer()

train_x_tokens = []
test_x_tokens = []

for x in train_x:
    train_x_tokens.append(tokenizer.tokenize(x))
    
for x in test_x:
    test_x_tokens.append(tokenizer.tokenize(x))

In [4]:
# define vocabulary using w266 common vocab (skipping cannonicalization for now)

train_text_all = [item for sublist in train_x_tokens for item in sublist]

vocab = vocabulary.Vocabulary(train_text_all, size=None)
print("Vocabulary size: {:,}".format(vocab.size))
print("Vocabulary dict: ", vocab.word_to_id)

train_x_ids = []
test_x_ids = []

for x in train_x_tokens:
    train_x_ids.append(vocab.words_to_ids(x))
    
for x in test_x_tokens:
    test_x_ids.append(vocab.words_to_ids(x))

Vocabulary size: 9,666
Vocabulary dict:  {'<s>': 0, '</s>': 1, '<unk>': 2, 'the': 3, ',': 4, '.': 5, 'to': 6, 'a': 7, 'and': 8, 'of': 9, 'I': 10, "'s": 11, 'that': 12, 'is': 13, 'in': 14, 'it': 15, "n't": 16, 'was': 17, 'for': 18, 'he': 19, '?': 20, 'be': 21, ')': 22, 'with': 23, '(': 24, 'you': 25, "''": 26, 'they': 27, 'have': 28, 'are': 29, 'but': 30, 'not': 31, 'do': 32, '``': 33, 'on': 34, ':': 35, '[': 36, ']': 37, 'as': 38, 'The': 39, 'just': 40, 'would': 41, 'this': 42, 'like': 43, 'she': 44, 'her': 45, ';': 46, 'show': 47, 'It': 48, 'all': 49, 'his': 50, 'or': 51, 'so': 52, 'about': 53, 'at': 54, 'has': 55, 'him': 56, 'from': 57, '!': 58, 'He': 59, 'if': 60, 'think': 61, 'more': 62, 'did': 63, 'people': 64, 'one': 65, 'books': 66, 'there': 67, 'That': 68, 'what': 69, 'http': 70, 'up': 71, 'by': 72, 'who': 73, 'will': 74, '&': 75, 'no': 76, 'And': 77, 'been': 78, 'does': 79, 'could': 80, 'know': 81, '...': 82, 'can': 83, 'when': 84, 'only': 85, 'them': 86, 'really': 87, 'becaus

In [5]:
# count token occurences and convert to feature vector for BoW model

train_x_fdict = []
test_x_fdict = []

for x in train_x_ids:
    train_x_fdict.append(collections.Counter(x))
    
for x in test_x_ids:
    test_x_fdict.append(collections.Counter(x))

train_x_vector = []
test_x_vector = []

num_features = vocab.size

for x in train_x_fdict:
    train_x_vector.append([x.get(i, 0) for i in range(num_features)])
    
for x in test_x_fdict:
    test_x_vector.append([x.get(i, 0) for i in range(num_features)])

# use w266 common utils to convert id lists to sparse bow matrix

train_x_sparse_bow = utils.id_lists_to_sparse_bow(train_x_fdict, vocab.size)
test_x_sparse_bow = utils.id_lists_to_sparse_bow(test_x_fdict, vocab.size)

# training Multinomial Naive Bayes for simple baseline model

nb = MultinomialNB()
nb.fit(train_x_sparse_bow, train_y)
y_pred = nb.predict(test_x_sparse_bow)

acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))

Accuracy on test set: 42.33%


In [6]:
# pad all id vectors to max length for Neural model

max_comment_length = max([len(x) for x in train_x_ids])

train_x_neural = train_x_ids
test_x_neural = test_x_ids

for x in train_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]
        
for x in test_x_neural:
    if len(x) < max_comment_length:
        for n in range(max_comment_length - len(x)):
            x.append(1)
    elif len(x) > max_comment_length:
        x = x[:max_comment_length]

# convert labels to one-hot vectors for Neural model

lb = preprocessing.LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)
lb.fit(np.asarray(train_y.tolist()).flatten())
    
train_y_neural = lb.transform(np.asarray(train_y.tolist()))
test_y_neural = lb.transform(np.asarray(test_y.tolist()))

In [7]:
# creating and training CNN for deep classification
# CNN model structure and code based on an implementation of Kim Yoon's "Convolutional Neural Networks for Sentence Classification"
# From https://github.com/dennybritz/cnn-text-classification-tf

# Parameters
# ==================================================

# Model Hyperparameters
tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
tf.flags.DEFINE_string("filter_sizes", "1,2,3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
tf.flags.DEFINE_integer("num_filters", 256, "Number of filters per filter size (default: 128)")
tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
tf.flags.DEFINE_integer("batch_size", 50, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS
FLAGS._parse_flags()
print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


# Data Preparation
# ==================================================

x_train = np.array(train_x_neural)
y_train = np.array(train_y_neural)
x_dev = np.array(test_x_neural)
y_dev = np.array(test_y_neural)

print("x_train", np.shape(x_train))
print("y_train", np.shape(y_train))
print("x_dev", np.shape(x_dev))
print("x_dev", np.shape(y_dev))

print("Vocabulary Size: {:d}".format(vocab.size))
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))


# Training
# ==================================================

def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]
            
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        cnn = TextCNN(
            sequence_length=x_train.shape[1],
            num_classes=10,
            vocab_size=vocab.size,
            embedding_size=FLAGS.embedding_dim,
            filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
            num_filters=FLAGS.num_filters,
            l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x: x_batch,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)

        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                print("\nEvaluation:")
                dev_step(x_dev, y_dev, writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))


Parameters:
ALLOW_SOFT_PLACEMENT=True
BATCH_SIZE=50
CHECKPOINT_EVERY=100
DROPOUT_KEEP_PROB=0.5
EMBEDDING_DIM=128
EVALUATE_EVERY=100
FILTER_SIZES=3,4,5
L2_REG_LAMBDA=0.0
LOG_DEVICE_PLACEMENT=False
NUM_CHECKPOINTS=5
NUM_EPOCHS=200
NUM_FILTERS=128

x_train [[  48   11    3 ...,    1    1    1]
 [4106 4107    1 ...,    1    1    1]
 [1702   84  102 ...,    1    1    1]
 ..., 
 [3478    4   30 ...,    1    1    1]
 [ 173 2777 4102 ...,    1    1    1]
 [ 108  785 1356 ...,    1    1    1]] (2700, 720)
y_train [[0 0 0 ..., 0 1 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [0 1 0 ..., 0 0 0]] (2700, 10)
x_dev [[  36    2 2246 ...,    1    1    1]
 [   2    2  108 ...,    1    1    1]
 [  68   41   21 ...,    1    1    1]
 ..., 
 [   2    2    1 ...,    1    1    1]
 [  68   13 1646 ...,    1    1    1]
 [ 338  155   11 ...,    1    1    1]] (300, 720)
x_dev [[0 0 0 ..., 0 1 0]
 [0 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 ..., 
 [0 0 1 ..., 0 0 0]
 [0 

2018-04-17T12:29:12.323064: step 86, loss 3.12458, acc 0.24
2018-04-17T12:29:12.762748: step 87, loss 3.12314, acc 0.22
2018-04-17T12:29:13.212685: step 88, loss 4.63871, acc 0.14
2018-04-17T12:29:13.658474: step 89, loss 3.43501, acc 0.24
2018-04-17T12:29:14.101748: step 90, loss 3.842, acc 0.18
2018-04-17T12:29:14.532688: step 91, loss 3.28611, acc 0.24
2018-04-17T12:29:14.978367: step 92, loss 3.56359, acc 0.18
2018-04-17T12:29:15.413135: step 93, loss 3.25887, acc 0.22
2018-04-17T12:29:15.856425: step 94, loss 4.16504, acc 0.1
2018-04-17T12:29:16.304405: step 95, loss 4.12287, acc 0.14
2018-04-17T12:29:16.741632: step 96, loss 4.82576, acc 0.08
2018-04-17T12:29:17.181899: step 97, loss 4.13379, acc 0.22
2018-04-17T12:29:17.611198: step 98, loss 3.90332, acc 0.12
2018-04-17T12:29:18.054350: step 99, loss 3.52063, acc 0.22
2018-04-17T12:29:18.502324: step 100, loss 3.87304, acc 0.22

Evaluation:
2018-04-17T12:29:19.510689: step 100, loss 2.11832, acc 0.253333

Saved model checkpoint 

2018-04-17T12:30:11.359457: step 216, loss 2.4287, acc 0.2
2018-04-17T12:30:11.793385: step 217, loss 2.37946, acc 0.4
2018-04-17T12:30:12.233165: step 218, loss 2.87388, acc 0.26
2018-04-17T12:30:12.675317: step 219, loss 2.42996, acc 0.3
2018-04-17T12:30:13.120376: step 220, loss 2.33821, acc 0.3
2018-04-17T12:30:13.565604: step 221, loss 1.95176, acc 0.48
2018-04-17T12:30:13.998315: step 222, loss 3.05846, acc 0.22
2018-04-17T12:30:14.434216: step 223, loss 2.90053, acc 0.2
2018-04-17T12:30:14.866657: step 224, loss 2.26615, acc 0.3
2018-04-17T12:30:15.306572: step 225, loss 2.75682, acc 0.32
2018-04-17T12:30:15.744409: step 226, loss 1.90119, acc 0.48
2018-04-17T12:30:16.176067: step 227, loss 2.6372, acc 0.28
2018-04-17T12:30:16.603326: step 228, loss 1.72869, acc 0.4
2018-04-17T12:30:17.024991: step 229, loss 2.86733, acc 0.2
2018-04-17T12:30:17.456139: step 230, loss 2.13385, acc 0.36
2018-04-17T12:30:17.887294: step 231, loss 2.46873, acc 0.28
2018-04-17T12:30:18.309456: step 2

2018-04-17T12:31:10.724357: step 349, loss 1.62224, acc 0.52
2018-04-17T12:31:11.168333: step 350, loss 1.91879, acc 0.48
2018-04-17T12:31:11.598888: step 351, loss 1.8409, acc 0.36
2018-04-17T12:31:12.035158: step 352, loss 1.66749, acc 0.5
2018-04-17T12:31:12.473451: step 353, loss 1.76736, acc 0.42
2018-04-17T12:31:12.901580: step 354, loss 2.01409, acc 0.4
2018-04-17T12:31:13.341030: step 355, loss 2.07589, acc 0.38
2018-04-17T12:31:13.777549: step 356, loss 1.31867, acc 0.54
2018-04-17T12:31:14.212320: step 357, loss 1.89846, acc 0.46
2018-04-17T12:31:14.651204: step 358, loss 1.69252, acc 0.46
2018-04-17T12:31:15.089797: step 359, loss 1.66253, acc 0.42
2018-04-17T12:31:15.530458: step 360, loss 2.12496, acc 0.36
2018-04-17T12:31:15.963659: step 361, loss 2.0844, acc 0.3
2018-04-17T12:31:16.400703: step 362, loss 1.65404, acc 0.42
2018-04-17T12:31:16.836110: step 363, loss 1.57334, acc 0.48
2018-04-17T12:31:17.270733: step 364, loss 2.01453, acc 0.44
2018-04-17T12:31:17.707117: s

2018-04-17T12:32:09.538276: step 481, loss 1.17897, acc 0.6
2018-04-17T12:32:09.978441: step 482, loss 1.63001, acc 0.5
2018-04-17T12:32:10.413071: step 483, loss 1.54703, acc 0.52
2018-04-17T12:32:10.844422: step 484, loss 1.36266, acc 0.58
2018-04-17T12:32:11.269463: step 485, loss 1.52484, acc 0.48
2018-04-17T12:32:11.714662: step 486, loss 1.34133, acc 0.48
2018-04-17T12:32:12.150037: step 487, loss 1.23307, acc 0.62
2018-04-17T12:32:12.584032: step 488, loss 1.51181, acc 0.5
2018-04-17T12:32:13.034057: step 489, loss 1.40373, acc 0.64
2018-04-17T12:32:13.477231: step 490, loss 0.974844, acc 0.72
2018-04-17T12:32:13.916562: step 491, loss 1.25265, acc 0.62
2018-04-17T12:32:14.345461: step 492, loss 1.01701, acc 0.6
2018-04-17T12:32:14.778210: step 493, loss 1.42061, acc 0.56
2018-04-17T12:32:15.226939: step 494, loss 0.990607, acc 0.68
2018-04-17T12:32:15.655963: step 495, loss 1.39129, acc 0.64
2018-04-17T12:32:16.106758: step 496, loss 1.57476, acc 0.58
2018-04-17T12:32:16.554664

...

2018-04-17T12:37:57.161428: step 1255, loss 0.375598, acc 0.84
2018-04-17T12:37:57.598951: step 1256, loss 0.274631, acc 0.92
2018-04-17T12:37:58.039831: step 1257, loss 0.396435, acc 0.86
2018-04-17T12:37:58.478173: step 1258, loss 0.336341, acc 0.92
2018-04-17T12:37:58.928746: step 1259, loss 0.415581, acc 0.88
2018-04-17T12:37:59.368366: step 1260, loss 0.308957, acc 0.92
2018-04-17T12:37:59.816305: step 1261, loss 0.424243, acc 0.86
2018-04-17T12:38:00.253996: step 1262, loss 0.305045, acc 0.92
2018-04-17T12:38:00.696140: step 1263, loss 0.415422, acc 0.88
2018-04-17T12:38:01.151622: step 1264, loss 0.306235, acc 0.96
2018-04-17T12:38:01.594640: step 1265, loss 0.353105, acc 0.88
2018-04-17T12:38:02.036318: step 1266, loss 0.336006, acc 0.9
2018-04-17T12:38:02.468269: step 1267, loss 0.278296, acc 0.96
2018-04-17T12:38:02.913715: step 1268, loss 0.230247, acc 0.94
2018-04-17T12:38:03.350327: step 1269, loss 0.277954, acc 0.92
2018-04-17T12:38:03.780752: step 1270, loss 0.285188, ac

2018-04-17T12:38:54.331699: step 1383, loss 0.244894, acc 0.96
2018-04-17T12:38:54.781898: step 1384, loss 0.336288, acc 0.92
2018-04-17T12:38:55.242719: step 1385, loss 0.304022, acc 0.9
2018-04-17T12:38:55.667472: step 1386, loss 0.266315, acc 0.96
2018-04-17T12:38:56.105018: step 1387, loss 0.27835, acc 0.92
2018-04-17T12:38:56.544772: step 1388, loss 0.326382, acc 0.94
2018-04-17T12:38:56.977729: step 1389, loss 0.280823, acc 0.96
2018-04-17T12:38:57.421413: step 1390, loss 0.269427, acc 0.94
2018-04-17T12:38:57.869927: step 1391, loss 0.251397, acc 0.94
2018-04-17T12:38:58.307644: step 1392, loss 0.24052, acc 0.92
2018-04-17T12:38:58.749973: step 1393, loss 0.420609, acc 0.9
2018-04-17T12:38:59.194698: step 1394, loss 0.145144, acc 0.98
2018-04-17T12:38:59.638113: step 1395, loss 0.404467, acc 0.94
2018-04-17T12:39:00.072445: step 1396, loss 0.268791, acc 0.96
2018-04-17T12:39:00.508505: step 1397, loss 0.301019, acc 0.96
2018-04-17T12:39:00.951083: step 1398, loss 0.231293, acc 0

2018-04-17T12:39:47.873709: step 1508, loss 0.279854, acc 0.88
2018-04-17T12:39:48.175951: step 1509, loss 0.291443, acc 0.92
2018-04-17T12:39:48.489728: step 1510, loss 0.259778, acc 0.9
2018-04-17T12:39:48.829817: step 1511, loss 0.282737, acc 0.94
2018-04-17T12:39:49.261948: step 1512, loss 0.190917, acc 1
2018-04-17T12:39:49.713584: step 1513, loss 0.193133, acc 0.96
2018-04-17T12:39:50.164400: step 1514, loss 0.148389, acc 0.96
2018-04-17T12:39:50.613186: step 1515, loss 0.237734, acc 0.98
2018-04-17T12:39:51.057657: step 1516, loss 0.139141, acc 0.98
2018-04-17T12:39:51.511137: step 1517, loss 0.179417, acc 0.98
2018-04-17T12:39:51.965874: step 1518, loss 0.280599, acc 0.94
2018-04-17T12:39:52.407268: step 1519, loss 0.297535, acc 0.94
2018-04-17T12:39:52.849025: step 1520, loss 0.230114, acc 0.94
2018-04-17T12:39:53.300702: step 1521, loss 0.237265, acc 0.94
2018-04-17T12:39:53.752783: step 1522, loss 0.264967, acc 0.96
2018-04-17T12:39:54.208158: step 1523, loss 0.381349, acc 0

2018-04-17T12:40:46.266837: step 1637, loss 0.222744, acc 0.94
2018-04-17T12:40:46.718910: step 1638, loss 0.276035, acc 0.9
2018-04-17T12:40:47.177862: step 1639, loss 0.208174, acc 0.98
2018-04-17T12:40:47.617019: step 1640, loss 0.246828, acc 0.92
2018-04-17T12:40:48.063666: step 1641, loss 0.0897128, acc 0.98
2018-04-17T12:40:48.517465: step 1642, loss 0.210035, acc 0.94
2018-04-17T12:40:48.978973: step 1643, loss 0.154819, acc 0.98
2018-04-17T12:40:49.441497: step 1644, loss 0.183751, acc 0.98
2018-04-17T12:40:49.890776: step 1645, loss 0.126109, acc 1
2018-04-17T12:40:50.345618: step 1646, loss 0.252372, acc 0.92
2018-04-17T12:40:50.787346: step 1647, loss 0.174154, acc 0.98
2018-04-17T12:40:51.239202: step 1648, loss 0.258981, acc 0.94
2018-04-17T12:40:51.699048: step 1649, loss 0.155882, acc 1
2018-04-17T12:40:52.138253: step 1650, loss 0.153577, acc 0.98
2018-04-17T12:40:52.576582: step 1651, loss 0.212528, acc 0.94
2018-04-17T12:40:53.023468: step 1652, loss 0.206226, acc 0.9

KeyboardInterrupt: 